# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [345]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
url = 'https://country-leaders.onrender.com'
# assign the /status endpoint to another variable called status_url (1 line)
status_url = 'https://country-leaders.onrender.com/status/'
# query the /status endpoint using the get() method and store it in the req variable (1 line)
r = requests.get('https://country-leaders.onrender.com/status/')
# check the status_code using a condition and print appropriate messages (4 lines)
if r.status_code == 200:
    print('Handshake confirmed')
else:
    print('Request failed with status code:', r.status_code)

Handshake confirmed


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [346]:
# Set the countries_url variable (1 line)
countries_url = "https://country-leaders.onrender.com/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
r = requests.get('https://country-leaders.onrender.com/countries')
# Get the JSON content and store it in the countries variable (1 line)
countries = r.json()
# display the request's status code and the countries variable (1 line)
print(r.status_code, countries)


403 {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [347]:
# Set the cookie_url variable (1 line)
cookie_url = 'https://country-leaders.onrender.com/cookie'
# Query the enpoint, set the cookies variable and display it (2 lines)
r = requests.get(cookie_url)
cookies = r.cookies
print(cookies)



<RequestsCookieJar[<Cookie user_cookie=4ff32852-cd03-4616-a41d-a5da0ae43c7d for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [348]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
r = requests.get(countries_url, cookies=cookies)
# display the countries variable (1 line)
countries_list = r.json()
print(countries_list)

['be', 'fr', 'ma', 'ru', 'us']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [349]:
# Set the leaders_url variable (1 line)
leaders_url = 'https://country-leaders.onrender.com/leaders'
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
r = requests.get(leaders_url, cookies=cookies)
# display the leaders variable (1 line)
leaders = r.json() 
print(leaders)


{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [350]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
Leaders_list = requests.get(leaders_url, cookies=cookies, params={'country': 'be'})
# assign the output to the leaders variable (1 line)
leaders = Leaders_list.json()

# display the leaders variable (1 line)
print(leaders)


[{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'https://nl

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [351]:
# 4 lines
# leaders_per_country = {}
# for country in countries_list:
#     r = requests.get(leaders_url, cookies=cookies, params={'country': country})
#     leaders_per_country[country] = r.json()

# print(leaders_per_country)
leaders_per_country = {}
for country in countries_list:
    # Query the /leaders endpoint using cookies and parameters for the current country
    r = requests.get(leaders_url, cookies=cookies, params={'country': country})
    
    # Assign the output to the leaders variable
    
    # Save the leaders data in the dictionary with the country code as the key
    leaders_per_country[country] = leaders

# Display the leaders_per_country variable
print(leaders_per_country)

    # leaders_per_country = leaders
    # print(leaders_per_country)
    
   


{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

In [352]:
# or 1 line
leaders_per_country = {country:requests.get(leaders_url, cookies=cookies, params={'country': country}).json() for country in countries_list}
print(leaders_per_country)


{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [353]:
import requests
# < 15 lines
def get_leaders():
    # URLs (you need to specify the actual URLs)
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    countries_url = "https://country-leaders.onrender.com/countries"
    leaders_url = 'https://country-leaders.onrender.com/leaders'

    # Get cookies
    r = requests.get(cookie_url)
    cookies = r.cookies

    # Get countries
    r = requests.get(countries_url, cookies=cookies)
    countries_list = r.json()

    leaders_per_country = {}
    
    # Loop through countries to get leaders
    for country in countries_list:
        r = requests.get(leaders_url, cookies=cookies, params={'country': country})
        leaders_per_country[country] = r.json()
        

    return leaders_per_country

    
    

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [354]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country)



{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [355]:
r = requests.get('https://nl.wikipedia.org/wiki/Jean-Luc_Dehaene')
r.text
print(r.text)
# 3 lines


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="nl" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Jean-Luc Dehaene - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [356]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, "html.parser")
soup_content = soup.prettify()
print(soup_content)


# 3 lines


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="nl">
 <head>
  <meta charset="utf-8"/>
  <title>
   Jean-Luc Dehaene - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-c

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [357]:
# 2 lines
soup = BeautifulSoup(r.text, "html.parser")
for tag in soup.find_all("p"):
    paragraphs = print(tag.get_text())


Jean Luc Joseph Marie Dehaene[5] uitspraakⓘ (Montpellier, 7 augustus 1940 – Quimper, 15 mei 2014) was van 1992 tot 1999 premier van België. Hij was lid van de toenmalige CVP (nu CD&V). Onder zijn regering werd België een volwaardige federale staat. Dehaene leidde twee coalitieregeringen met de socialisten en bekleedde verschillende ministerposten voor hij in 1992 eerste minister werd. Van 2000 tot 2007 was hij burgemeester van Vilvoorde. In 2002 werd hij voorgedragen als vicevoorzitter van de Europese Conventie.

Dehaene werd in 1940 geboren in de noodkliniek van het Franse Montpellier, waar zijn familie in het begin van de Tweede Wereldoorlog een toevlucht had gevonden. Hij was de zoon van de Brugse psychiater Albert Dehaene en Andrée Verstraete. Datzelfde jaar keerde hij echter terug naar Brugge, waar hij het grootste deel van zijn jeugd doorbracht.

Dehaene doorliep gedeeltelijk de klassieke humaniora aan het door de paters jezuïeten geleide Sint-Jozefscollege te Aalst. Aan de unive

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [358]:
# <10 lines
# Getting the first paragraph
# first_paragraph = soup.find('p') 
# if first_paragraph:
#     print(first_paragraph.get_text())
# else:
#     print("No paragraph found.")

for tag in soup.find_all("p"):
    first_paragraph = soup.find('p').getText()
    break
print(first_paragraph)



Jean Luc Joseph Marie Dehaene[5] uitspraakⓘ (Montpellier, 7 augustus 1940 – Quimper, 15 mei 2014) was van 1992 tot 1999 premier van België. Hij was lid van de toenmalige CVP (nu CD&V). Onder zijn regering werd België een volwaardige federale staat. Dehaene leidde twee coalitieregeringen met de socialisten en bekleedde verschillende ministerposten voor hij in 1992 eerste minister werd. Van 2000 tot 2007 was hij burgemeester van Vilvoorde. In 2002 werd hij voorgedragen als vicevoorzitter van de Europese Conventie.



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [359]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)  # keep this for the rest of the notebook
 
    #   [insert your code]
    r = requests.get(wikipedia_url)
    soup = BeautifulSoup(r.text, "html.parser")   

    first_paragraph = soup.find('p').get_text()  # Get text from the first paragraph
    return first_paragraph 


In [360]:
# Test: 3 lines
def test_get_first_paragraph():
    url = 'https://nl.wikipedia.org/wiki/Jean-Luc_Dehaene'  # Example Wikipedia page
    result = get_first_paragraph(url)  # Call the function
    print( result)  # Print the result
test_get_first_paragraph()

https://nl.wikipedia.org/wiki/Jean-Luc_Dehaene
Jean Luc Joseph Marie Dehaene[5] uitspraakⓘ (Montpellier, 7 augustus 1940 – Quimper, 15 mei 2014) was van 1992 tot 1999 premier van België. Hij was lid van de toenmalige CVP (nu CD&V). Onder zijn regering werd België een volwaardige federale staat. Dehaene leidde twee coalitieregeringen met de socialisten en bekleedde verschillende ministerposten voor hij in 1992 eerste minister werd. Van 2000 tot 2007 was hij burgemeester van Vilvoorde. In 2002 werd hij voorgedragen als vicevoorzitter van de Europese Conventie.



### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [361]:
# 3 lines
import re
first_paragraph_sanitzed = re.sub(r'\[\d+\]', '', first_paragraph)
print(first_paragraph_sanitzed)





Jean Luc Joseph Marie Dehaene uitspraakⓘ (Montpellier, 7 augustus 1940 – Quimper, 15 mei 2014) was van 1992 tot 1999 premier van België. Hij was lid van de toenmalige CVP (nu CD&V). Onder zijn regering werd België een volwaardige federale staat. Dehaene leidde twee coalitieregeringen met de socialisten en bekleedde verschillende ministerposten voor hij in 1992 eerste minister werd. Van 2000 tot 2007 was hij burgemeester van Vilvoorde. In 2002 werd hij voorgedragen als vicevoorzitter van de Europese Conventie.



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [362]:
# 10 lines
import re
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)  # keep this for the rest of the notebook
 
    #   [insert your code]
    r = requests.get(wikipedia_url)
    soup = BeautifulSoup(r.text, "html.parser")   

    first_paragraph = soup.find('p').get_text()  # Get text from the first paragraph
    first_paragraph_sanitzed = re.sub(r'\[\d+\]', '', first_paragraph)
   
    return first_paragraph_sanitzed
print(first_paragraph_sanitzed) 



Jean Luc Joseph Marie Dehaene uitspraakⓘ (Montpellier, 7 augustus 1940 – Quimper, 15 mei 2014) was van 1992 tot 1999 premier van België. Hij was lid van de toenmalige CVP (nu CD&V). Onder zijn regering werd België een volwaardige federale staat. Dehaene leidde twee coalitieregeringen met de socialisten en bekleedde verschillende ministerposten voor hij in 1992 eerste minister werd. Van 2000 tot 2007 was hij burgemeester van Vilvoorde. In 2002 werd hij voorgedragen als vicevoorzitter van de Europese Conventie.



Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [363]:
# < 20 lines
import re
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)  # keep this for the rest of the notebook
 
    #   [insert your code]
    r = requests.get(wikipedia_url)
    soup = BeautifulSoup(r.text, "html.parser")   

    first_paragraph = soup.find('p').get_text()  # Get text from the first paragraph
    first_paragraph_sanitzed = re.sub(r'\[\d+\]', '', first_paragraph)
    first_paragraph_sanitzed = re.sub(r'<.*?>', '', first_paragraph)
    first_paragraph_sanitzed = re.sub(r'\s+', '', first_paragraph).strip()
    first_paragraph_sanitzed = re.sub(r'\xa0', '', first_paragraph)
   
    return first_paragraph_sanitzed
print(first_paragraph_sanitzed) 


Jean Luc Joseph Marie Dehaene uitspraakⓘ (Montpellier, 7 augustus 1940 – Quimper, 15 mei 2014) was van 1992 tot 1999 premier van België. Hij was lid van de toenmalige CVP (nu CD&V). Onder zijn regering werd België een volwaardige federale staat. Dehaene leidde twee coalitieregeringen met de socialisten en bekleedde verschillende ministerposten voor hij in 1992 eerste minister werd. Van 2000 tot 2007 was hij burgemeester van Vilvoorde. In 2002 werd hij voorgedragen als vicevoorzitter van de Europese Conventie.



## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [376]:
# < 20 lines

def get_leaders():
    # URLs (you need to specify the actual URLs)
    cookie_url = 'https://country-leaders.onrender.com/cookie'
    countries_url = "https://country-leaders.onrender.com/countries"
    leaders_url = 'https://country-leaders.onrender.com/leaders'

    # Get cookies
    r = requests.get(cookie_url)
    cookies = r.cookies

    # Get countries
    r = requests.get(countries_url, cookies=cookies)
    countries_list = r.json()

    leaders_per_country = {}
    
    # Loop through countries to get leaders
    for country in countries_list:
        r = requests.get(leaders_url, cookies=cookies, params={'country': country})
        leaders_per_country[country] = r.json()
        for leader in leaders:
            r = requests.get(leaders_url['wikipedia_url'], cookies= cookies, params={'country': country})
            get_first_paragraph(r)


    return leaders_per_country
print(leaders_per_country)



{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

In [365]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [366]:
# < 25 lines



Check the output of your function again.

In [367]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [368]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [369]:
# <25 lines


Test your new functions.



## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [370]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [371]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [372]:
# 3 lines


In [373]:
# Call the function (1 line)


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!